In [ ]:
from sympy import Matrix, Symbol, simplify, symbols


def build_functional_matrix(n, k) -> Matrix:
    f = Matrix(symbols(",".join(f"f{j}" for j in range(1, n + 1))))
    gammas = symbols(",".join(f"\gamma{i}" for i in range(1, k + 1)))
    lambdas = symbols(",".join(f"\lambda{i}" for i in range(1, k + 1)))
    result = Matrix.zeros(rows=n, cols=1)
    if k == 1:
        gammas = [gammas]
        lambdas = [lambdas]
    for gamma, lambd in zip(gammas, lambdas):
        result += gamma * Matrix([lambd**j for j in range(n)])

    diff: Matrix = f - result
    return diff


def build_functional(n, k):
    diff = build_functional_matrix(n, k)
    return simplify(diff.norm() ** 2)


In [ ]:
import numpy as np
from sklearn.metrics import pairwise_distances


def restruct_f(f_vec: np.ndarray) -> np.ndarray:
    distances = pairwise_distances(f_vec.reshape(-1, 1), metric="l1")
    x = np.argmax(np.abs(f_vec))
    order = [x]
    for _ in range(len(f_vec) - 1):
        distances[:, x] = -np.inf
        x = np.argmax(distances[x])
        order.append(x)

    # print(np.unravel_index(np.argmax(distances), distances.shape))
    # print(np.argmax(pairwise_distances(f_vec.reshape(-1, 1))))
    return f_vec[order[::-1]]


In [ ]:
from scipy.optimize import minimize

n = 6
while True:
    f_vec = np.random.randint(-3, 3, n) + np.random.random(n)
    # f_vec = np.array(
    #     [1.94714385, -2.06999027, -1.77805665, -1.47865507, 1.45323721, -1.59903824]
    # )
    # f_vec = np.array([0.5489, 0.0641, -0.3059, 0.8434, -0.853, -0.3352])
    f_vec = np.array([1, -2, 1])
    # f_vec = restruct_f(f_vec)
    n = len(f_vec)
    # 6 2 10.226062424586425 gamma=[0.5823549  0.58430421] lambda=[-0.79111583 -1.17055549]
    # print(f_vec)
    for k in range(1, n):
        J = build_functional(n, k)
        # display(J)
        expr = J.subs({Symbol(f"f{i}"): f_vec[i - 1] for i in range(1, n + 1)})

        def func(x):
            return float(
                expr.subs(
                    {Symbol(f"\gamma{i}"): x[i - 1] for i in range(1, k + 1)}
                    | {Symbol(f"\lambda{i}"): x[i + k - 1] for i in range(1, k + 1)}
                ).evalf(n=21)
            )

        b_gamma = [-10.0, 10.0]
        b_lambda = [-1000.0, 0]
        bounds = [b_gamma] * k + [b_lambda] * k

        x0 = [-1] * 2 * k
        solution = minimize(func, x0, bounds=bounds)  # method="L-BFGS-B", tol=1e-9)
        errors = [
            np.sum(solution.x[:k] * solution.x[k:] ** i) - f_vec[i] for i in range(0, n)
        ]
        print(
            n,
            k,
            solution.fun,
            f"gamma={solution.x[:k]}",
            f"lambda={solution.x[k:]}",
        )
        print(errors)

        # print(n, k, solution.fun, f"gamma={solution.x[:k]}", f"lambda={solution.x[k:]}")
        # if k > 1 and np.mean(((solution.x[k]) - solution.x[k + 1]) ** 2) > 1e-2:
        #     print(f_vec)
        #     print(
        #         n,
        #         k,
        #         solution.fun,
        #         f"gamma={solution.x[:k]}",
        #         f"lambda={solution.x[k:]}",
        #     )
        if k == 1:
            left, right = 0, 0
            gamma_1 = float(solution.x[0])
            lambda_1 = float(solution.x[1])
            print(gamma_1, lambda_1)
            for i in range(0, n, 2):
                # print(
                #     f"нечетные_{i + 1} = {2 * f_vec[i] - gamma_1 * (lambda_1 ** (i))}",
                #     # f"четные_{i+1} = {2 * f_vec[i + 1] - gamma_1 * (lambda_1 ** (i + 1))}",
                # )
                left += 2 * f_vec[i] - gamma_1 * (lambda_1**i)
                if i + 1 < n:
                    right += 2 * f_vec[i + 1] - gamma_1 * (lambda_1 ** (i + 1))
            print(f"нечетные = {left}", f" четные = {right}")


In [ ]:
0.666666666666667 * 2 - 1.333

In [ ]:
import numpy as np
from IPython.display import Math, display
from scipy.optimize import minimize
from sympy.printing import latex
from toolz import compose

mega_print = compose(display, Math) if False else print

n = 25

f_vec = np.round(2 * np.random.random(n) - 1, 4)
# f_vec = np.array(
#     [1.94714385, -2.06999027, -1.77805665, -1.47865507, 1.45323721, -1.59903824]
# )
# f_vec = np.array([1, 1, -2])
f_vec = restruct_f(f_vec)
n = len(f_vec)
mega_print(f"F={latex(Matrix(f_vec))}")
# f_vec = [1.94714385, -2.06999027, -1.77805665, -1.47865507, 1.45323721, -1.59903824]
# 6 2 10.226062424586425 gamma=[0.5823549  0.58430421] lambda=[-0.79111583 -1.17055549]
# print(f_vec)


def recurs(expression, k, gammas, lambdas):
    expression = expression.subs(
        {Symbol(f"\gamma{i + 1}"): gamma for i, gamma in enumerate(gammas)}
        | {Symbol(f"\lambda{i + 1}"): lambd for i, lambd in enumerate(lambdas)}
    )
    # display(expression)

    def func(x):
        return float(
            expression.subs(
                {Symbol(f"\gamma{k}"): x[0]} | {Symbol(f"\lambda{k}"): x[1]}
            ).evalf(n=21)
        )

    b_gamma = [-10.0, 10.0]
    b_lambda = [-1000.0, 0]
    bounds = [b_gamma] + [b_lambda]
    x0 = [-1] * 2
    # print(func(x0))
    solution = minimize(func, x0, bounds=bounds)
    # print(
    #     n,
    #     k,
    #     solution.fun,
    #     f"gamma{k}={solution.x[0]}",
    #     f"lambda{k}={solution.x[1]}",
    # )
    mega_print(f"\min{{J_{k}^{n}}}={np.round(solution.fun, 4)}")
    mega_print(f"\gamma_{k}={np.round(solution.x[0], 4)}")
    mega_print(f"\lambda_{k}={np.round(solution.x[1], 4)}")
    return solution.x[0], solution.x[1]


gammas_array, lambdas_array = [], []

for k in range(1, n + 1):
    J = build_functional(n, k)
    # display(J)
    expr = J.subs({Symbol(f"f{i}"): f_vec[i - 1] for i in range(1, n + 1)})
    g, l = recurs(expr, k, gammas_array, lambdas_array)
    gammas_array.append(g)
    lambdas_array.append(l)


In [ ]:
Matrix(f_vec)

 build_functional_matrix(n, k+1)